In [ ]:
!pip uninstall tensorflow

In [ ]:
!pip install tensorflow

## 1st WINDOW

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QPushButton, QVBoxLayout, QWidget
from PyQt5.QtGui import QFont
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap
from detections import EmotionDetector  # Import the detection window class

class WelcomeWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.initUI()

    def initUI(self):
        self.setWindowTitle('Welcome Window')
        self.setGeometry(100, 100, 800, 600)

        # Background image
        bg_label = QLabel(self)
        bg_label.setGeometry(0, 0, 800, 600)  # Set geometry to cover the entire window
        pixmap = QPixmap("background_image22.jpg")  # Replace "background_image22.jpg" with the path to your image
        scaled_pixmap = pixmap.scaled(800, 600)  # Scale pixmap to fit the window size
        bg_label.setPixmap(scaled_pixmap)

        # Central widget
        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        # Layout
        layout = QVBoxLayout()

        # Welcome label
        self.welcome_label = QLabel('Welcome!', self)
        font = QFont('Calibri', 48)
        font.setItalic(True)  # Make the font italic
        self.welcome_label.setFont(font)
        self.welcome_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(self.welcome_label)

        # Quote label
        quote_label = QLabel('In every emotion lies a story waiting to be told.', self)
        quote_font = QFont('Calibri', 16, italic=True, weight=QFont.Bold)  # Define font for the quote label with bold weight
        quote_label.setFont(quote_font)
        quote_label.setAlignment(Qt.AlignCenter)
        quote_label.setStyleSheet("color: black;")  # Set text color to blue using CSS style
        layout.addWidget(quote_label)


        # Open button
        open_button = QPushButton('Open', self)
        open_button.setFont(QFont('Calibri', 24))
        open_button.clicked.connect(self.open_action)
        open_button.setStyleSheet("background-color: #4CAF50; color: white; font-size: 28px; font-family: Arial; font-style: italic; border-radius: 10px;")
        layout.addWidget(open_button)

        # Set layout
        central_widget.setLayout(layout)

    def open_action(self):
        print("Open button clicked")
        # Create and show the detection window when the "Open" button is clicked
        self.detection_window = EmotionDetector()
        self.detection_window.show()


def main():
    app = QApplication(sys.argv)
    window = WelcomeWindow()
    window.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()


ModuleNotFoundError: No module named 'tensorflow.tsl'

## 2ND WINDOW

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QVBoxLayout
from PyQt5.QtCore import QTimer
from PyQt5.QtGui import QImage, QPixmap
from suggestions import SuggestionsApp
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
import pygame
import os
import random
from collections import Counter

class EmotionDetector(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Emotion Detector")
        self.setGeometry(100, 100, 1200, 600)

        # Background image
        bg_label = QLabel(self)
        bg_label.setGeometry(0, 0, 1200, 600)  # Set geometry to cover the entire window
        pixmap = QPixmap("background_image.jpg")  # Replace "background_image.jpg" with the path to your image
        bg_label.setPixmap(pixmap)

        self.start_button = QPushButton('Start Detection', self)
        self.start_button.setGeometry(100, 100, 150, 50)
        self.start_button.clicked.connect(self.start_detection)
        self.start_button.setStyleSheet("background-color: #4CAF50; color: white; font-size: 18px; font-family: Arial; font-style: italic; border-radius: 10px;")

        self.stop_button = QPushButton('Stop Detection', self)
        self.stop_button.setGeometry(300, 100, 150, 50)
        self.stop_button.clicked.connect(self.stop_detection)
        self.stop_button.setEnabled(False)
        self.stop_button.setStyleSheet("background-color: #4CAF50; color: white; font-size: 18px; font-family: Arial; font-style: italic; border-radius: 10px;")

        self.detected_emotion_label = QLabel("", self)
        self.detected_emotion_label.setGeometry(100, 200, 200, 30)
        self.detected_emotion_label.setStyleSheet("QLabel { color: red; font-weight: bold; font-size: 18px; font-family: Arial; }")

        self.video_label = QLabel(self)
        self.video_label.setGeometry(300, 250, 400, 231)

        self.play_music_button = QPushButton('Play Music', self)
        self.play_music_button.setGeometry(500, 100, 150, 50)
        self.play_music_button.clicked.connect(self.play_music)
        self.play_music_button.setEnabled(False)
        self.play_music_button.setStyleSheet("background-color: #4CAF50; color: white; font-size: 18px; font-family: Arial; font-style: italic; border-radius: 10px;")

        self.open_suggestions_button = QPushButton('Open Suggestions', self)
        self.open_suggestions_button.setGeometry(700, 100, 150, 50)
        self.open_suggestions_button.clicked.connect(self.open_suggestions_window)
        self.open_suggestions_button.setStyleSheet("background-color: #4CAF50; color: white; font-size: 18px; font-family: Arial; font-style: italic; border-radius: 10px;")

        self.mode_emotion_label = QLabel("", self)
        self.mode_emotion_label.setGeometry(300, 500, 400, 30)
        self.mode_emotion_label.setStyleSheet("QLabel { color: red; font-weight: bold; font-size: 18px; font-family: Arial; font-style: italic; }")


        self.timer = QTimer(self)
        self.timer.timeout.connect(self.detect_emotion)

        self.face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.classifier = load_model(r'model.h5')
        self.detected_emotions = []
        self.emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Disinterested', 'Sad', 'Surprise']
        self.myemo = None
        self.cap = cv2.VideoCapture(0)

        # Directory where the folders are located
        self.base_directory = r'C:\Users\Aditya\ONLY PROJECTS\Emotion_22 - Copy\Music-moods'

        self.your_gallery_button = QPushButton('Your Gallery', self)
        self.your_gallery_button.setGeometry(900, 100, 150, 50)
        self.your_gallery_button.clicked.connect(self.open_gallery)
        self.your_gallery_button.setStyleSheet("background-color: #4CAF50; color: white; font-size: 18px; font-family: Arial; font-style: italic; border-radius: 10px;")

    def open_gallery(self):
        os.system("python mygallery.py")

    def start_detection(self):
        self.start_button.setEnabled(False)
        self.stop_button.setEnabled(True)
        self.play_music_button.setEnabled(False)
        self.open_suggestions_button.setEnabled(False)
        self.timer.start(100)

    def stop_detection(self):
        self.start_button.setEnabled(True)
        self.stop_button.setEnabled(False)
        self.play_music_button.setEnabled(True)
        self.open_suggestions_button.setEnabled(True)
        self.timer.stop()
        self.cap.release()  # Release the video capture object

        # Show mode of detected emotions
        self.show_mode_emotion()

    def detect_emotion(self):
        _, frame = self.cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB format for displaying in PyQt
        height, width, channel = frame.shape
        step = channel * width
        qImg = QImage(frame.data, width, height, step, QImage.Format_RGB888)
        self.video_label.setPixmap(QPixmap.fromImage(qImg))

        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        faces = self.face_classifier.detectMultiScale(gray)

        # Only process the first detected face
        if len(faces) == 1:
            (x, y, w, h) = faces[0]  # Get the coordinates of the first detected face
            roi_gray = gray[y:y + h, x:x + w]
            roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

            if np.sum([roi_gray]) != 0:
                roi = roi_gray.astype('float') / 255.0
                roi = img_to_array(roi)
                roi = np.expand_dims(roi, axis=0)

                prediction = self.classifier.predict(roi)[0]
                label = self.emotion_labels[prediction.argmax()]

                self.detected_emotion_label.setText(label)
                self.detected_emotions.append(label)
                self.myemo = label
            else:
                self.detected_emotion_label.setText('No Faces')
        else:
            # If no face is detected or more than one face is detected, show a message
            self.detected_emotion_label.setText('Please show only one face')

    def play_music(self):
        if self.myemo is not None:
            folder_path = os.path.join(self.base_directory, self.myemo)
            music_files = [file for file in os.listdir(folder_path) if file.endswith('.mp3')]
            if not music_files:
                print(f"No music files found in {folder_path}")
                return
            random_music_file = random.choice(music_files)
            music_file_path = os.path.join(folder_path, random_music_file)

            pygame.init()
            pygame.mixer.init()
            try:
                pygame.mixer.music.load(music_file_path)
                pygame.mixer.music.play()
                # Add a delay to allow the audio to play
                pygame.time.delay(10000)  # 10000 milliseconds (10 seconds)
                pygame.mixer.music.stop()
            finally:
                pygame.quit()
        else:
            print("No emotion detected to play music for.")

    def open_suggestions_window(self):
        if self.myemo is not None:
            self.suggestions_window = SuggestionsApp(self.myemo)  # Pass detected emotion to SuggestionsApp
            self.suggestions_window.show()
        else:
            print("No emotion detected.")

    def show_mode_emotion(self):
        if self.detected_emotions:
            mode_emotion = Counter(self.detected_emotions).most_common(1)[0][0]
            self.myemo = mode_emotion  # Set myemo to the mode emotion
            self.mode_emotion_label.setText(f"Mode Emotion: {mode_emotion}")
        else:
            self.mode_emotion_label.setText("No emotions detected yet.")



if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = EmotionDetector()
    window.show()
    sys.exit(app.exec_())


## 3RD WINDOW

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLineEdit, QPushButton, QListWidget, QListWidgetItem
from PyQt5.QtCore import Qt, QUrl
from PyQt5.QtGui import QDesktopServices
from googlesearch import search
from pytube import Search
import requests
import random

class ClickableListWidgetItem(QListWidgetItem):
    def __init__(self, text, url):
        super().__init__(text)
        self.url = url
        

class SuggestionsApp(QWidget):
    def __init__(self, detected_emotion):
        super().__init__()
        self.setWindowTitle("Suggestions App")
        self.detected_emotion = detected_emotion
        self.setup_ui()
        self.resize(800, 600) 

    def setup_ui(self):
        layout = QVBoxLayout()

        # Query input field
        self.query_input = QLineEdit()
        layout.addWidget(self.query_input)

        # Buttons for search
        search_buttons_layout = QHBoxLayout()
        self.blog_button = QPushButton("Search Blogs")
        self.blog_button.clicked.connect(self.search_blogs)
        search_buttons_layout.addWidget(self.blog_button)
        self.blog_button.setStyleSheet("background-color: #4CAF50; color: white; font-size: 18px; font-family: Arial; font-style: italic; border-radius: 10px;")

        self.video_button = QPushButton("Search Videos")
        self.video_button.clicked.connect(self.search_videos)
        self.video_button.setStyleSheet("background-color: #4CAF50; color: white; font-size: 18px; font-family: Arial; font-style: italic; border-radius: 10px;")
        search_buttons_layout.addWidget(self.video_button)

        self.book_button = QPushButton("Search Books")
        self.book_button.clicked.connect(self.search_books)
        self.book_button.setStyleSheet("background-color: #4CAF50; color: white; font-size: 18px; font-family: Arial; font-style: italic; border-radius: 10px;")
        search_buttons_layout.addWidget(self.book_button)

        layout.addLayout(search_buttons_layout)

        # List widgets to display search results
        self.blog_list = QListWidget()
        self.blog_list.itemClicked.connect(self.open_link)
        layout.addWidget(self.blog_list)

        self.video_list = QListWidget()
        self.video_list.itemClicked.connect(self.open_link)
        layout.addWidget(self.video_list)

        self.book_list = QListWidget()
        self.book_list.itemClicked.connect(self.open_link)
        layout.addWidget(self.book_list)

        self.setLayout(layout)

        # Set detected emotion as initial text in query input field
        if self.detected_emotion:
            self.query_input.setText(self.detected_emotion)

    def search_blogs(self):
        self.blog_list.clear()
        query = self.query_input.text()
        google_query = f"i am feeling {query} suggest me some blogs"
        for i, result in enumerate(search(google_query, tld="com", num=5, stop=5, pause=2), start=1):
            item = ClickableListWidgetItem(f"Blog {i}: {result}", result)
            item.setText(f"Blog {i}: {result}")
            self.blog_list.addItem(item)

    def search_videos(self):
        self.video_list.clear()
        query = self.query_input.text()
        youtube_query = f"i am feeling {query} suggest me some videos"
        search_results = Search(youtube_query)
        for i, result in enumerate(search_results.results[:5], start=1):
            item = ClickableListWidgetItem(f"Video {i}: {result.title} - {result.watch_url}", result.watch_url)
            item.setText(f"Video {i}: {result.title} - {result.watch_url}")
            self.video_list.addItem(item)

    def search_books(self):
        self.book_list.clear()
        query = self.query_input.text()
        books_query = f"books about {query}"
        book_results = self.get_books(books_query)
        if book_results:
            random_books = random.sample(book_results, min(5, len(book_results)))
            for i, book in enumerate(random_books, start=1):
                title = book["volumeInfo"]["title"]
                link = book["volumeInfo"]["infoLink"]
                item = ClickableListWidgetItem(f"Book {i}: {title} - {link}", link)
                item.setText(f"Book {i}: {title} - {link}")
                self.book_list.addItem(item)
        else:
            self.book_list.addItem("No books found.")

    def get_books(self, query):
        url = f"https://www.googleapis.com/books/v1/volumes?q={query}"
        response = requests.get(url)
        data = response.json()
        if "items" in data:
            return data["items"]
        else:
            return []

    def open_link(self, item):
        url = item.url
        QDesktopServices.openUrl(QUrl(url))

if __name__ == "__main__":
    app = QApplication(sys.argv)
    suggestions_app = SuggestionsApp("happy")  # For testing, replace "happy" with the detected emotion
    suggestions_app.show()
    sys.exit(app.exec_())


## gallery

In [ ]:
import os
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QFileDialog, QVBoxLayout, QHBoxLayout, QPushButton, QWidget, QLabel, QListWidget
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt

class PictureGallery(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Picture Gallery")
        self.setFixedSize(800, 600)  # Set maximum size for the window

        self.initUI()

    def initUI(self):
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        self.layout = QHBoxLayout()
        self.central_widget.setLayout(self.layout)

        # List to hold the paths of the images
        self.image_paths = []

        # List widget to display image names
        self.list_widget = QListWidget()
        self.list_widget.itemClicked.connect(self.preview_image)
        self.layout.addWidget(self.list_widget)

        # Button to open a folder containing images
        self.open_button = QPushButton("Open Folder")
        self.open_button.clicked.connect(self.open_folder)
        self.layout.addWidget(self.open_button)

        # Label to display the preview of the selected image
        self.preview_label = QLabel()
        self.preview_label.setScaledContents(True)
        self.preview_label.setFixedSize(400, 400)  # Set fixed size for the preview label
        self.layout.addWidget(self.preview_label)

    def open_folder(self):
        folder_path = QFileDialog.getExistingDirectory(self, "Open Folder")
        if folder_path:
            self.image_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith((".jpg", ".jpeg", ".png"))]
            self.list_widget.addItems([os.path.basename(file) for file in self.image_paths])

    def preview_image(self, item):
        index = self.list_widget.currentRow()
        if index >= 0:
            image_path = self.image_paths[index]
            pixmap = QPixmap(image_path)
            self.preview_label.setPixmap(pixmap)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    gallery = PictureGallery()
    gallery.show()
    sys.exit(app.exec_())
